Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-data-dependency-steps.png)

# Azure Machine Learning Data Prep
In this notebook, we will see how we can do data prep using the azureml.dataprep SDK.

### Azure Machine Learning and Pipeline SDK-specific Imports

In [1]:
# Core ML to create exeperiments and run them
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

# AzureML Dataprep SDK
import azureml.dataprep as dprep

# Check core SDK version number
print("AML Core SDK version:", azureml.core.VERSION)

AML Core SDK version: 1.0.83


### Getting AML Workspace and Compute

In [2]:
ws = Workspace.from_config()
print("== Workspace:")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure blob storage)
# def_blob_store = ws.get_default_datastore()
blob_store = Datastore(ws, "workspaceblobstore")
print("== Datastore: {}".format(blob_store.name))

# list compute targets
print("== Compute targets:")
for ct in ws.compute_targets:
    print("  " + ct)
    
# Retrieve a compute target    
from azureml.core.compute_target import ComputeTargetException
aml_compute_target = "agd-training-cpu"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("== AML compute target attached: " + aml_compute_target)
except ComputeTargetException:
    print("== AML compute target not found: " + aml_compute_target)

== Workspace:
agd-mlws
azure-ml-workshop
westus2
c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60
== Datastore: workspaceblobstore
== Compute targets:
  agd-inference
  agd-inference-v
  agd-training-gpu
  agd-training-cpu
== AML compute target attached: agd-training-cpu


### Data inputs definitions

In [3]:
# inputs definitions
h_ts_1_dr = DataReference(datastore=blob_store,data_reference_name="h_ts_1",path_on_datastore="datasets/time-series/h-time-series-1.csv")
h_ts_2_dr = DataReference(datastore=blob_store,data_reference_name="h_ts_2",path_on_datastore="datasets/time-series/h-time-series-2.csv")
h_ts_3_dr = DataReference(datastore=blob_store,data_reference_name="h_ts_3",path_on_datastore="datasets/time-series/h-time-series-3.csv")
d_ts_1_dr = DataReference(datastore=blob_store,data_reference_name="d_ts_1",path_on_datastore="datasets/time-series/d-time-series-1.csv")
d_ts_2_dr = DataReference(datastore=blob_store,data_reference_name="d_ts_2",path_on_datastore="datasets/time-series/d-time-series-2.csv")

print("== Datasets defined")

== Datasets defined


## Data prep using Azure ML Dataprep SDK

In [4]:
# read data as azureml.dataprep.DataFlow objects
h_ts_1_dflow = dprep.read_csv(h_ts_1_dr)
h_ts_2_dflow = dprep.read_csv(h_ts_2_dr)
h_ts_3_dflow = dprep.read_csv(h_ts_3_dr)
d_ts_1_dflow = dprep.read_csv(d_ts_1_dr)
d_ts_2_dflow = dprep.read_csv(d_ts_2_dr)

In [5]:
h_ts_1_dflow.head(6)

,MYDATE,HOUR,NODE_ID,MW
0,1/1/2012,1,N1,2.78994586
1,1/1/2012,1,S1,-3.860853503
2,1/1/2012,1,Z1,-0.675159236
3,1/1/2012,2,N1,2.785633758
4,1/1/2012,2,S1,-3.788318471
5,1/1/2012,2,Z1,-0.675159236


In [6]:
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.STRING,1/1/2012,9/9/2015,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.STRING,1,9,100799.0,0.0,100799.0,0.0,0.0,0.0,25,,,,,,,,,,,,,,,,
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.STRING,-0.00033121,999.9626592,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,


In [7]:
d_ts_1_dflow.head(6)

,RDATE,X1
0,1/1/2012,59.8
1,1/2/2012,62
2,1/3/2012,62
3,1/4/2012,63.4
4,1/5/2012,55.4
5,1/6/2012,54.8


In [8]:
d_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
RDATE,FieldType.STRING,1/1/2012,9/9/2015,1521.0,0.0,1521.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
X1,FieldType.STRING,100.4,99.8,1521.0,0.0,1521.0,0.0,0.0,0.0,245,,,,,,,,,,,,,,,,


### Handling Data Types

In [9]:
# this is a good technique assuming you're not sure about the data types (for this example, it doesn't necessary
# make sense as you could quickly understand the format of each field, especially the 'date' one)

# discover data types
builder = h_ts_1_dflow.builders.set_column_types()
builder.learn()
builder.conversion_candidates

{'MYDATE': [
     (FieldType.DATE, ['%d/%m/%Y']),
     (FieldType.DATE, ['%m/%d/%Y'])],
 'HOUR': [FieldType.INTEGER],
 'NODE_ID': [FieldType.STRING],
 'MW': [FieldType.DECIMAL]}

In [10]:
# resolve MYDATE type ambiguity
builder.conversion_candidates['MYDATE'] = (dprep.FieldType.DATE, ['%m/%d/%Y'])
h_ts_1_dflow = builder.to_dataflow()
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.INTEGER,1,25,100799.0,0.0,100799.0,0.0,0.0,0.0,25,1,3,3,6.28018,12.0127,18.0145,23,24,24,12.4607,6.90726,47.7102,0.0036578,-1.20297,25,1
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.DECIMAL,-32.7938,2437.81,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,-10.5711,-3.65979,-3.76832,-1.14616,7.83233,314.028,1608.31,2131.88,2370.15,274.534,506.661,256705,2.3246,4.63076,669.44,-32.7938


### Explicit type setup

In [11]:
# IF you know what types should be up front, you can force types to the dataflow
h_ts_1_dflow = dprep.read_csv(h_ts_1_dr)
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.STRING,1/1/2012,9/9/2015,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.STRING,1,9,100799.0,0.0,100799.0,0.0,0.0,0.0,25,,,,,,,,,,,,,,,,
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.STRING,-0.00033121,999.9626592,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,


In [12]:
# setting data types explicitely
h_ts_1_dflow = h_ts_1_dflow.to_datetime('MYDATE', ['%m/%d/%Y'])
h_ts_1_dflow = h_ts_1_dflow.to_long(['HOUR'])
h_ts_1_dflow = h_ts_1_dflow.to_number(['MW'])
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.INTEGER,1,25,100799.0,0.0,100799.0,0.0,0.0,0.0,25,1,3,3,6.28018,12.0127,18.0145,23,24,24,12.4607,6.90726,47.7102,0.0036578,-1.20297,25,1
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.DECIMAL,-32.7938,2437.81,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,-10.5711,-3.65979,-3.76832,-1.14616,7.83233,314.028,1608.31,2131.88,2370.15,274.534,506.661,256705,2.3246,4.63076,669.44,-32.7938


### Automatic data type detection with 'auto_read_file'

In [13]:
# instead 'auto_read_file' will do type auto detect (when it can), and other things (auto skip first rows, will find the right header, etc)
h_ts_1_dflow = dprep.auto_read_file(h_ts_1_dr)
d_ts_1_dflow = dprep.auto_read_file(d_ts_1_dr)

In [14]:
# Note: potential ambiguity may prevent correct detection of data types (we'll handle down below): MYDATE
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.STRING,1/1/2012,9/9/2015,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.INTEGER,1,25,100799.0,0.0,100799.0,0.0,0.0,0.0,25,1,3,3,6.28018,12.0127,18.0145,23,24,24,12.4607,6.90726,47.7102,0.0036578,-1.20297,25,1
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.DECIMAL,-32.7938,2437.81,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,-10.5711,-3.65979,-3.76832,-1.14616,7.83233,314.028,1608.31,2131.88,2370.15,274.534,506.661,256705,2.3246,4.63076,669.44,-32.7938


In [15]:
# All types auto detected correctly
d_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
RDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,1521.0,0.0,1521.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
X1,FieldType.DECIMAL,50.4,106.8,1521.0,0.0,1521.0,0.0,0.0,0.0,245,50.6462,62.1737,62.0222,69.172,78.166,88.0893,95.7833,100.633,105.983,78.1219,11.5587,133.604,-0.0766716,-0.901754,106.8,50.4


### Summary of how to best read data to benefit from auto type detection and auto skipping info lines to find header, etc.

In [16]:
# =======================
# azureml-dataprep-sdk.py (WIP)
# =======================

h_ts_1_dflow = dprep.auto_read_file(h_ts_1_dr)
h_ts_2_dflow = dprep.auto_read_file(h_ts_2_dr)
h_ts_3_dflow = dprep.auto_read_file(h_ts_3_dr)
d_ts_1_dflow = dprep.auto_read_file(d_ts_1_dr)
d_ts_2_dflow = dprep.auto_read_file(d_ts_2_dr)

# ===========================
# EOF azureml-dataprep-sdk.py (WIP)
# ===========================

### Pivoting Data

In [17]:
# ts_1 and ts_2 need to be pivoted
h_ts_1_dflow.head(6)

,MYDATE,HOUR,NODE_ID,MW
0,1/1/2012,1,N1,2.789946
1,1/1/2012,1,S1,-3.860854
2,1/1/2012,1,Z1,-0.675159
3,1/1/2012,2,N1,2.785634
4,1/1/2012,2,S1,-3.788318
5,1/1/2012,2,Z1,-0.675159


In [18]:
# PIVOT data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_1_pivot_dflow.head(2)

,MYDATE,HOUR,N1,S1,Z1
0,1/1/2012,1,2.789946,-3.860854,-0.675159
1,1/1/2012,2,2.785634,-3.788318,-0.675159


### Data transformation by Example

In [19]:
# using derive_column_by_example to reformat DATE/HOUR as a DATETIME field
builder = h_ts_1_pivot_dflow.builders.derive_column_by_example(source_columns = ['MYDATE','HOUR'], new_column_name = 'DATETIME')
builder.add_example(source_data = {'MYDATE': '1/1/2012', 'HOUR': 1}, example_value = '01/01/2012 01:00')
builder.add_example(source_data = {'MYDATE': '10/10/2012', 'HOUR': 15}, example_value = '10/10/2012 15:00')
builder.add_example(source_data = {'MYDATE': '1/17/2012', 'HOUR': 12}, example_value = '01/17/2012 12:00')
builder.preview(skip=3000,count=10)

,MYDATE,HOUR,DATETIME
0,5/14/2012,7,05/14/2012 07:00
1,5/14/2012,8,05/14/2012 08:00
2,5/14/2012,9,05/14/2012 09:00
3,5/14/2012,10,05/14/2012 10:00
4,5/14/2012,11,05/14/2012 11:00
5,5/14/2012,12,05/14/2012 12:00
6,5/14/2012,13,05/14/2012 13:00
7,5/14/2012,14,05/14/2012 14:00
8,5/14/2012,15,05/14/2012 15:00
9,5/14/2012,16,05/14/2012 16:00


In [20]:
h_ts_1_pivot_dt_dflow = builder.to_dataflow()
h_ts_1_pivot_dt_dflow.head(6)

,MYDATE,HOUR,DATETIME,N1,S1,Z1
0,1/1/2012,1,01/01/2012 01:00,2.789946,-3.860854,-0.675159
1,1/1/2012,2,01/01/2012 02:00,2.785634,-3.788318,-0.675159
2,1/1/2012,3,01/01/2012 03:00,2.786000,-3.887990,-0.676115
3,1/1/2012,4,01/01/2012 04:00,2.785965,-4.119672,-0.700669
4,1/1/2012,5,01/01/2012 05:00,2.790325,-4.828436,-0.702354
5,1/1/2012,6,01/01/2012 06:00,2.788650,-5.092978,-0.706051


In [21]:
h_ts_1_pivot_dt_dflow.dtypes

MYDATE                   FieldType.STRING
HOUR                     FieldType.INTEGER
DATETIME                 FieldType.STRING
N1                       FieldType.DECIMAL
S1                       FieldType.DECIMAL
Z1                       FieldType.DECIMAL

In [22]:
# NOTE: we're keeping the 'MYDATE' column in place as it'll simplify the summarization at the day level we'll do later on
# we're building a 'DATETIME' column to later on join the hourly datasets (even though we could join them on the two existing
# DATE and HOUR columns), but also as an interesting exercise to use 'derive_column_by_example'
h_ts_1_pivot_dt_dflow = h_ts_1_pivot_dflow.derive_column_by_example(
    source_columns = ['MYDATE','HOUR'],
    new_column_name = 'DATETIME',
    example_data = [({'MYDATE': '1/1/2012', 'HOUR': '1'},    '01/01/2012 01:00'),
                    ({'MYDATE': '10/10/2012', 'HOUR': '15'}, '10/10/2012 15:00'),
                    ({'MYDATE': '1/17/2012', 'HOUR': '12'},  '01/17/2012 12:00')]
    ).drop_columns(['HOUR'])
h_ts_1_pivot_dt_dflow.head(6)

,MYDATE,DATETIME,N1,S1,Z1
0,1/1/2012,01/01/2012 01:00,2.789946,-3.860854,-0.675159
1,1/1/2012,01/01/2012 02:00,2.785634,-3.788318,-0.675159
2,1/1/2012,01/01/2012 03:00,2.786000,-3.887990,-0.676115
3,1/1/2012,01/01/2012 04:00,2.785965,-4.119672,-0.700669
4,1/1/2012,01/01/2012 05:00,2.790325,-4.828436,-0.702354
5,1/1/2012,01/01/2012 06:00,2.788650,-5.092978,-0.706051


In [23]:
builder = h_ts_1_pivot_dt_dflow.builders.set_column_types()
builder.learn()
builder.conversion_candidates

{'MYDATE': [
     (FieldType.DATE, ['%d/%m/%Y']),
     (FieldType.DATE, ['%m/%d/%Y'])], 'DATETIME': [
     (FieldType.DATE, ['%d/%m/%Y %H:%M']),
     (FieldType.DATE, ['%m/%d/%Y %H:%M'])], 'N1': [FieldType.DECIMAL], 'S1': [FieldType.DECIMAL], 'Z1': [FieldType.DECIMAL]}

In [24]:
builder.conversion_candidates['MYDATE'] = (dprep.FieldType.DATE, ['%m/%d/%Y'])
builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%m/%d/%Y %H:%M'])
h_ts_1_pivot_dt_dflow = builder.to_dataflow()
h_ts_1_pivot_dt_dflow.head(6)

,MYDATE,DATETIME,N1,S1,Z1
0,2012-01-01,2012-01-01 01:00:00,2.789946,-3.860854,-0.675159
1,2012-01-01,2012-01-01 02:00:00,2.785634,-3.788318,-0.675159
2,2012-01-01,2012-01-01 03:00:00,2.786000,-3.887990,-0.676115
3,2012-01-01,2012-01-01 04:00:00,2.785965,-4.119672,-0.700669
4,2012-01-01,2012-01-01 05:00:00,2.790325,-4.828436,-0.702354
5,2012-01-01,2012-01-01 06:00:00,2.788650,-5.092978,-0.706051


In [25]:
h_ts_1_pivot_dt_dflow.dtypes

MYDATE                   FieldType.DATE  
DATETIME                 FieldType.DATE  
N1                       FieldType.DECIMAL
S1                       FieldType.DECIMAL
Z1                       FieldType.DECIMAL

In [26]:
h_ts_1_pivot_dt_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,,,,,,,,,,,,,,,,
DATETIME,FieldType.DATE,2012-01-01 01:00:00+00:00,2016-02-29 23:00:00+00:00,33600.0,4.0,33596.0,0.000119,0.0,0.0,>1000,,,,,,,,,,,,,,,,
N1,FieldType.DECIMAL,-32.7938,473.713,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-7.02568,-1.00306,-1.00727,-0.738948,5.28022,196.082,391.086,433.924,452.942,102.114,137.342,18862.9,1.0419,-0.32315,473.713,-32.7938
S1,FieldType.DECIMAL,-14.5467,2437.81,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-10.9195,-7.45157,-7.49409,-5.27086,27.9864,1280.7,2024.96,2282.61,2396.81,592.049,755.079,570144,0.825234,-0.860473,2437.81,-14.5467
Z1,FieldType.DECIMAL,-2.30379,563.445,33600.0,1.0,33599.0,0.000030,0.0,0.0,>1000,-2.11192,-1.61922,-1.62392,-1.35039,4.72356,270.328,468.624,522.22,553.707,129.436,171.8,29515.2,0.93803,-0.638352,563.445,-2.30379


### WIP version of our dataprep code

In [27]:
# =======================
# azureml-dataprep-sdk.py
# =======================

h_ts_1_dflow = dprep.auto_read_file(h_ts_1_dr)
h_ts_2_dflow = dprep.auto_read_file(h_ts_2_dr)
h_ts_3_dflow = dprep.auto_read_file(h_ts_3_dr)
d_ts_1_dflow = dprep.auto_read_file(d_ts_1_dr)
d_ts_2_dflow = dprep.auto_read_file(d_ts_2_dr)

# Pivot data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_2_pivot_dflow = h_ts_2_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])

# merge DATE and HOUR and update datatype for DATETIME column
def ts_merge_date_hour_to_datetime(dflow, date_column_name, hour_column_name):
    # merge columns
    dflow = dflow.derive_column_by_example(
        source_columns = [date_column_name,hour_column_name],
        new_column_name = 'DATETIME',
        example_data = [({date_column_name: '1/1/2012', hour_column_name: '1'},    '01/01/2012 01:00'),
                        ({date_column_name: '10/10/2012', hour_column_name: '15'}, '10/10/2012 15:00'),
                        ({date_column_name: '1/17/2012', hour_column_name: '12'},  '01/17/2012 12:00')]
        ).drop_columns([hour_column_name])
    # update data type
    builder = dflow.builders.set_column_types()
    builder.learn()
    builder.conversion_candidates[date_column_name] = (dprep.FieldType.DATE, ['%m/%d/%Y'])
    builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%m/%d/%Y %H:%M'])
    return builder.to_dataflow()

# generate all DATETIME columns with proper data type
h_ts_1_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_1_pivot_dflow,'MYDATE','HOUR')
h_ts_2_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_2_pivot_dflow,'MYDATE','HOUR')
h_ts_3_dt_dflow =       ts_merge_date_hour_to_datetime(h_ts_3_dflow,      'DATE','HE')

# ===========================
# EOF azureml-dataprep-sdk.py (WIP)
# ===========================

In [28]:
# check profile
h_ts_1_pivot_dt_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,,,,,,,,,,,,,,,,
DATETIME,FieldType.DATE,2012-01-01 01:00:00+00:00,2016-02-29 23:00:00+00:00,33600.0,4.0,33596.0,0.000119,0.0,0.0,>1000,,,,,,,,,,,,,,,,
N1,FieldType.DECIMAL,-32.7938,473.713,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-7.02568,-1.00306,-1.00727,-0.738948,5.28022,196.082,391.086,433.924,452.942,102.114,137.342,18862.9,1.0419,-0.32315,473.713,-32.7938
S1,FieldType.DECIMAL,-14.5467,2437.81,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-10.9195,-7.45157,-7.49409,-5.27086,27.9864,1280.7,2024.96,2282.61,2396.81,592.049,755.079,570144,0.825234,-0.860473,2437.81,-14.5467
Z1,FieldType.DECIMAL,-2.30379,563.445,33600.0,1.0,33599.0,0.000030,0.0,0.0,>1000,-2.11192,-1.61922,-1.62392,-1.35039,4.72356,270.328,468.624,522.22,553.707,129.436,171.8,29515.2,0.93803,-0.638352,563.445,-2.30379


In [29]:
# check profile
d_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
RDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,1521.0,0.0,1521.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
X1,FieldType.DECIMAL,50.4,106.8,1521.0,0.0,1521.0,0.0,0.0,0.0,245,50.6462,62.1737,62.0222,69.172,78.166,88.0893,95.7833,100.633,105.983,78.1219,11.5587,133.604,-0.0766716,-0.901754,106.8,50.4


### Joining Data sets

In [30]:
# JOINING DATA SET h_ts_j1=h1,h2
h_ts_j1_dflow = dprep.Dataflow.join(
    h_ts_1_pivot_dt_dflow,
    h_ts_2_pivot_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='h1_',
    right_column_prefix='h2_'
).drop_columns(['h2_MYDATE','h2_DATETIME']).rename_columns({'h1_MYDATE':'MYDATE','h1_DATETIME':'DATETIME'})
h_ts_j1_dflow.head(5)

,MYDATE,DATETIME,h1_N1,h1_S1,h1_Z1,h2_N1,h2_S1
0,2012-01-01,2012-01-01 01:00:00,2.789946,-3.860854,-0.675159,214.201076,76.939389
1,2012-01-01,2012-01-01 02:00:00,2.785634,-3.788318,-0.675159,208.534389,66.742051
2,2012-01-01,2012-01-01 03:00:00,2.786000,-3.887990,-0.676115,188.653331,70.460099
3,2012-01-01,2012-01-01 04:00:00,2.785965,-4.119672,-0.700669,192.039484,76.141580
4,2012-01-01,2012-01-01 05:00:00,2.790325,-4.828436,-0.702354,204.381194,79.607589


In [31]:
# JOINING DATA SET h_ts_dflow=h_ts_j1,h3
h_ts_dflow = dprep.Dataflow.join(
    h_ts_j1_dflow,
    h_ts_3_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='',
    right_column_prefix='h3_'
).drop_columns(['h3_DATE','h3_DATETIME'])
h_ts_dflow.head(5)

,MYDATE,DATETIME,h1_N1,h1_S1,h1_Z1,h2_N1,h2_S1,h3_CLOAD
0,2012-01-01,2012-01-01 01:00:00,2.789946,-3.860854,-0.675159,214.201076,76.939389,7458.280255
1,2012-01-01,2012-01-01 02:00:00,2.785634,-3.788318,-0.675159,208.534389,66.742051,7214.649682
2,2012-01-01,2012-01-01 03:00:00,2.786000,-3.887990,-0.676115,188.653331,70.460099,7018.789809
3,2012-01-01,2012-01-01 04:00:00,2.785965,-4.119672,-0.700669,192.039484,76.141580,6912.420382
4,2012-01-01,2012-01-01 05:00:00,2.790325,-4.828436,-0.702354,204.381194,79.607589,6913.057325


### WIP version of our dataprep code so far

In [32]:
# =======================
# azureml-dataprep-sdk.py (WIP)
# =======================

h_ts_1_dflow = dprep.auto_read_file(h_ts_1_dr)
h_ts_2_dflow = dprep.auto_read_file(h_ts_2_dr)
h_ts_3_dflow = dprep.auto_read_file(h_ts_3_dr)
d_ts_1_dflow = dprep.auto_read_file(d_ts_1_dr)
d_ts_2_dflow = dprep.auto_read_file(d_ts_2_dr)

# Pivot data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_2_pivot_dflow = h_ts_2_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])

# merge DATE and HOUR and update datatype for DATETIME column
def ts_merge_date_hour_to_datetime(dflow, date_column_name, hour_column_name):
    # merge columns
    dflow = dflow.derive_column_by_example(
        source_columns = [date_column_name,hour_column_name],
        new_column_name = 'DATETIME',
        example_data = [({date_column_name: '1/1/2012', hour_column_name: '1'},    '01/01/2012 01:00'),
                        ({date_column_name: '10/10/2012', hour_column_name: '15'}, '10/10/2012 15:00'),
                        ({date_column_name: '1/17/2012', hour_column_name: '12'},  '01/17/2012 12:00')]
        ).drop_columns([hour_column_name])
    # update data type
    builder = dflow.builders.set_column_types()
    builder.learn()
    builder.conversion_candidates[date_column_name] = (dprep.FieldType.DATE, ['%m/%d/%Y'])
    builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%m/%d/%Y %H:%M'])
    return builder.to_dataflow()

# generate all DATETIME columns with proper data type
h_ts_1_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_1_pivot_dflow,'MYDATE','HOUR')
h_ts_2_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_2_pivot_dflow,'MYDATE','HOUR')
h_ts_3_dt_dflow =       ts_merge_date_hour_to_datetime(h_ts_3_dflow,      'DATE','HE')

# JOINING DATA SET h_ts_dflow=join(h1,h2,h3)
h_ts_dflow = dprep.Dataflow.join(
    dprep.Dataflow.join(h_ts_1_pivot_dt_dflow,
                        h_ts_2_pivot_dt_dflow,
                        join_key_pairs=[('DATETIME', 'DATETIME')],
                        left_column_prefix='h1_',right_column_prefix='h2_'
                       ).drop_columns(['h2_MYDATE','h2_DATETIME']).rename_columns({'h1_MYDATE':'MYDATE','h1_DATETIME':'DATETIME'}),
    h_ts_3_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='',
    right_column_prefix='h3_'
).drop_columns(['h3_DATE','h3_DATETIME'])

# JOINING DATA SET d_ts_dflow=join(d1,d2)
d_ts_dflow = dprep.Dataflow.join(
    d_ts_1_dflow,
    d_ts_2_dflow,
    join_key_pairs=[('RDATE', 'RDATE')],
    left_column_prefix='',
    right_column_prefix='r_').drop_columns(['r_RDATE']).rename_columns({'r_X2':'X2'})

# ===========================
# EOF azureml-dataprep-sdk.py (WIP)
# ===========================

In [33]:
# check profile
d_ts_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
RDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,1521.0,0.0,1521.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
X1,FieldType.DECIMAL,50.4,106.8,1521.0,0.0,1521.0,0.0,0.0,0.0,245,50.6462,62.1737,62.0222,69.172,78.166,88.0893,95.7833,100.633,105.983,78.1219,11.5587,133.604,-0.0766716,-0.901754,106.8,50.4
X2,FieldType.DECIMAL,0,250845,1521.0,0.0,1521.0,0.0,0.0,0.0,>1000,32.8185,8180.23,7911.43,22444.1,41917.2,60445.5,109850,196398,245758,46666,35988.2,1.29515e+09,1.92288,6.20864,115663,0


In [34]:
# check profile 
h_ts_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,,,,,,,,,,,,,,,,
DATETIME,FieldType.DATE,2012-01-01 01:00:00+00:00,2016-02-29 23:00:00+00:00,33653.0,64.0,33589.0,0.001902,0.0,0.0,>1000,,,,,,,,,,,,,,,,
h1_N1,FieldType.DECIMAL,-32.7938,473.713,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-7.01141,-1.00289,-1.0049,-0.738381,4.7042,196.121,390.774,433.882,452.929,101.897,137.272,18843.7,1.04504,-0.316437,473.713,-32.7938
h1_S1,FieldType.DECIMAL,-14.5467,2437.81,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-10.9219,-7.45055,-7.49119,-5.2657,26.8337,1279.15,2026.29,2281.04,2396.56,590.765,754.705,569580,0.828503,-0.855033,2437.81,-14.5467
h1_Z1,FieldType.DECIMAL,-2.30379,563.445,33653.0,1.0,33652.0,0.000030,0.0,0.0,>1000,-2.11183,-1.6188,-1.62755,-1.34989,4.5957,269.651,468.544,521.809,554.027,129.157,171.709,29483.8,0.941151,-0.632476,563.445,-2.30379
h2_N1,FieldType.DECIMAL,-3.65543,458.068,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-2.7496,1.67475,1.5734,17.3363,81.8513,219.794,363.396,413.888,438.416,125.987,123.462,15242.9,0.770255,-0.67567,458.068,-3.65543
h2_S1,FieldType.DECIMAL,-5.92685,1143.99,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-3.94448,26.6717,25.9997,70.8251,244.402,521.081,814.267,968.584,1078.06,315.286,268.397,72037,0.669233,-0.646651,1143.99,-5.92685
h3_CLOAD,FieldType.DECIMAL,4950.32,15894.6,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,5771.88,6541.17,6529.08,7076.07,7948.11,8787.34,11415.6,13170.1,14450.9,8178.58,1518.85,2.3069e+06,1.29312,1.95259,11309.6,4950.32


### Summarizing hourly data to daily

In [35]:
# summarize h_ts_dflow to daily
def generate_summary_column(column_name,column_suffix,summary_function):
    return dprep.SummaryColumnsValue(
                column_id=column_name,
                summary_column_name=column_name+'_'+column_suffix,
                summary_function=summary_function)

def generate_summary_columns(dflow):
    summary_columns = []
    for key in h_ts_dflow.get_profile().columns.keys():
        if key != 'MYDATE' and key != 'DATETIME':
            summary_columns.append(generate_summary_column(key,'MAX',dprep.SummaryFunction.MAX))
            summary_columns.append(generate_summary_column(key,'MIN',dprep.SummaryFunction.MIN))
            summary_columns.append(generate_summary_column(key,'MEAN',dprep.SummaryFunction.MEAN))
            summary_columns.append(generate_summary_column(key,'MEDIAN',dprep.SummaryFunction.MEDIAN))
    return summary_columns

h_ts_summarized_dflow = h_ts_dflow.summarize(
    summary_columns=generate_summary_columns(h_ts_dflow),
    group_by_columns=['MYDATE'],
    join_back=False)

h_ts_summarized_dflow.head(4)

,MYDATE,h1_N1_MAX,h1_N1_MIN,h1_N1_MEAN,h1_N1_MEDIAN,h1_S1_MAX,h1_S1_MIN,h1_S1_MEAN,h1_S1_MEDIAN,h1_Z1_MAX,...,h2_N1_MEAN,h2_N1_MEDIAN,h2_S1_MAX,h2_S1_MIN,h2_S1_MEAN,h2_S1_MEDIAN,h3_CLOAD_MAX,h3_CLOAD_MIN,h3_CLOAD_MEAN,h3_CLOAD_MEDIAN
0,2012-01-01,77.964809,2.776551,21.051649,3.091959,783.017506,-5.092978,222.621629,-3.729854,228.803038,...,91.617776,92.262092,79.607589,32.513153,51.655914,45.377755,8648.726115,6912.420382,7469.662144,7235.987261
1,2012-01-02,55.436481,3.040615,15.320979,3.084408,568.235076,-5.019124,153.208645,-3.756280,210.288054,...,23.196309,22.833188,87.108025,32.716647,59.034386,62.987688,8713.694268,6604.458599,7497.895320,7326.751592
2,2012-01-03,70.889146,3.056631,17.834587,3.153653,493.581704,-4.856099,125.727916,-3.735742,140.487076,...,18.935850,18.291605,249.495041,53.387736,137.928525,143.000892,8788.853503,6448.407643,7332.414844,7118.152866
3,2012-01-04,39.108446,3.045873,10.545203,3.152172,281.776172,-4.759627,60.874459,-3.723787,153.746898,...,32.246362,4.398803,144.913226,18.507299,49.848768,39.022344,9407.324841,6308.598726,7978.011631,8285.668790


In [36]:
h_ts_summarized_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
h1_N1_MAX,FieldType.DECIMAL,-1.04211,473.713,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-0.919621,92.9141,92.9141,187.567,266.448,366.609,435.059,451.472,470.646,261.127,122.059,14898.5,-0.391051,-0.583275,473.713,-1.04211
h1_N1_MIN,FieldType.DECIMAL,-32.7938,7.91993,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-11.5086,-1.14324,-1.14448,-1.03392,-0.819506,-0.563383,0.92885,3.17949,7.21483,-0.662073,1.34198,1.80091,-9.37838,231.367,0.0983457,-1.59163
h1_N1_MEAN,FieldType.DECIMAL,-1.04211,216.837,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-0.919621,20.1795,20.1795,55.9374,88.2134,139.906,194.144,209.546,216.536,95.5194,56.1388,3151.57,0.203564,-0.780106,216.837,-1.04211
h1_N1_MEDIAN,FieldType.DECIMAL,-10.7138,231.293,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-1.16002,-0.850363,-0.856266,-0.59386,4.10531,58.5485,161.104,190.998,203.929,35.0798,52.1428,2718.87,1.52575,1.3622,146.693,-10.7138
h1_S1_MAX,FieldType.DECIMAL,-9.05072,2437.81,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-8.63159,780.399,751.413,1339.54,1616.75,2015.6,2320.93,2399.84,2430.05,1564.32,591.614,350007,-1.08856,0.984294,2437.81,449.78
h1_S1_MIN,FieldType.DECIMAL,-14.5467,16.8798,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-14.393,-8.95739,-8.97564,-8.1452,-6.87572,-3.96793,-1.2179,3.41376,12.9617,-6.06113,3.00728,9.04376,1.3251,4.11493,1.91641,-14.4093
h1_S1_MEAN,FieldType.DECIMAL,-9.05072,1057.36,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-8.63159,206.189,189.262,413.541,562.5,740.21,953.499,1012.65,1047.78,553.941,255.41,65234.5,-0.3935,-0.267577,1057.36,-9.05072
h1_S1_MEDIAN,FieldType.DECIMAL,-10.204,781.353,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-9.85716,-6.8024,-6.83799,-2.84959,28.6368,272.201,590.398,688.812,762.836,148.798,203.337,41345.9,1.20022,0.215925,682.492,-10.204
h1_Z1_MAX,FieldType.DECIMAL,-1.62555,563.445,1505.0,0.0,1505.0,0.0,0.0,0.0,>1000,-1.56783,166.223,156.606,291.913,378.888,463.764,526.188,552.693,562.103,357.454,139.679,19510.3,-1.02009,0.616074,563.445,44.316


## Extra Featurization on X1 and X2

### Working with Quantiles

In [37]:
# leverage the profile metadata for the data set, with a ColumnProfile for each column

profile = d_ts_dflow.get_profile()
columns = profile.columns

print(f"\nColumn Profile:{columns['X2']}")
print(f"\nQuantiles for X2:{columns['X2'].quantiles}")
print(f"\n5% Quantile for X2:{columns['X2'].quantiles[0.05]}")
print(f"\n95% Quantile for X2:{columns['X2'].quantiles[0.95]}")


Column Profile:ColumnProfile:
    column_name: X2
    type: FieldType.DECIMAL

    min: 0.0
    max: 250844.586
    count: 1521.0
    missing_count: 0.0
    not_missing_count: 1521.0
    percent_missing: 0.0
    error_count: 0.0
    empty_count: 0.0
    unique_values: >1000


    Quantiles (est.):
         0.1%: 32.81847133448
           1%: 8180.228844465713
           5%: 7911.4251592499995
          25%: 22444.050994708625
          50%: 41917.202349230756
          75%: 60445.47031359524
          95%: 109849.66558588893
          99%: 196398.03740324985
        99.9%: 245757.63693440022

    mean: 46665.98114717059
    std: 35988.24225665022
    variance: 1295153580.7233446
    skewness: 1.9228822892849444
    kurtosis: 6.208639001801183
    whisker_top: 115663.05732857143
    whisker_bottom: 0.0


Quantiles for X2:OrderedDict([(0.001, 32.81847133448), (0.01, 8180.228844465713), (0.05, 7911.4251592499995), (0.25, 22444.050994708625), (0.5, 41917.202349230756), (0.75, 60445.470313

In [38]:
# Grabbing quantile information as variables for further featurization of X2
X2_column_profile = d_ts_dflow.get_profile().columns['X2']
X2_quantile_005 = X2_column_profile.quantiles[0.05]
X2_quantile_095 = X2_column_profile.quantiles[0.95]

# mapping function to update the dataflow, this gets executed in parallel for each dataflow partition
def update_X2_with_quantiles(df, index):
    df.loc[df['X2'] < X2_quantile_005, 'X2'] = X2_quantile_005
    df.loc[df['X2'] > X2_quantile_095, 'X2'] = X2_quantile_095
    #df['X2_original'] = df['X2']
    return df

d_ts_features_dflow = d_ts_dflow.map_partition(fn=update_X2_with_quantiles)
d_ts_features_dflow.head(5)

,RDATE,X1,X2
0,2012-01-01,59.8,47365.60510
1,2012-01-02,62.0,48127.07006
2,2012-01-03,62.0,54601.27389
3,2012-01-04,63.4,74656.36943
4,2012-01-05,55.4,87156.68790


### Generating new column for 'TEMP' of X1

In [39]:
# we can use custom script to code any type of logic to generate new columns, but this is slower than the 'map_partition
# above which can run in parallel when multiple partitions are applied
d_ts_features_dflow = d_ts_features_dflow.new_script_column(new_column_name='X1_TEMP', insert_after='X1', script="""
def newvalue(row):
    value = 'MEDIUM'
    if row['X1'] > 80:
        value = 'HOT'
    if row['X1'] < 60:
        value = 'COLD'
    return value
""")
d_ts_features_dflow.head(5)

,RDATE,X1,X1_TEMP,X2
0,2012-01-01,59.8,COLD,47365.60510
1,2012-01-02,62.0,MEDIUM,48127.07006
2,2012-01-03,62.0,MEDIUM,54601.27389
3,2012-01-04,63.4,MEDIUM,74656.36943
4,2012-01-05,55.4,COLD,87156.68790


### One Hot Encoding of 'X1_TEMP'

In [40]:
d_ts_features_dflow = d_ts_features_dflow.one_hot_encode(source_column='X1_TEMP')
d_ts_features_dflow.head(5)

,RDATE,X1,X1_TEMP,X1_TEMP_MEDIUM,X1_TEMP_HOT,X1_TEMP_COLD,X2
0,2012-01-01,59.8,COLD,0,0,1,47365.60510
1,2012-01-02,62.0,MEDIUM,1,0,0,48127.07006
2,2012-01-03,62.0,MEDIUM,1,0,0,54601.27389
3,2012-01-04,63.4,MEDIUM,1,0,0,74656.36943
4,2012-01-05,55.4,COLD,0,0,1,87156.68790


### Lag Variables on X2

In [41]:
# mapping function to update the dataflow with Lags
def update_X2_with_lags(df, index):
    df['X2_lag_1'] = df['X2'].shift(-1)
    df['X2_lag_7'] = df['X2'].shift(-7)
    df['X2_rolling_7'] = df['X2'].rolling(7).mean()
    return df
d_ts_features_dflow = d_ts_features_dflow.map_partition(fn=update_X2_with_lags)
d_ts_features_dflow.head(10)

,RDATE,X1,X1_TEMP,X1_TEMP_MEDIUM,X1_TEMP_HOT,X1_TEMP_COLD,X2,X2_lag_1,X2_lag_7,X2_rolling_7
0,2012-01-01,59.8,COLD,0,0,1,47365.60510,48127.07006,89002.86624,NaN
1,2012-01-02,62.0,MEDIUM,1,0,0,48127.07006,54601.27389,59700.95541,NaN
2,2012-01-03,62.0,MEDIUM,1,0,0,54601.27389,74656.36943,55801.91083,NaN
3,2012-01-04,63.4,MEDIUM,1,0,0,74656.36943,87156.68790,64844.58599,NaN
4,2012-01-05,55.4,COLD,0,0,1,87156.68790,100811.78340,69787.89809,NaN
5,2012-01-06,54.8,COLD,0,0,1,100811.78340,75496.49682,37906.68790,NaN
6,2012-01-07,55.0,COLD,0,0,1,75496.49682,89002.86624,44400.00000,69745.040943
7,2012-01-08,58.6,COLD,0,0,1,89002.86624,59700.95541,49031.21019,75693.221106
8,2012-01-09,57.6,COLD,0,0,1,59700.95541,55801.91083,50068.15287,77346.633299
9,2012-01-10,58.8,COLD,0,0,1,55801.91083,64844.58599,52394.58599,77518.152861


### Impute missing values

In [42]:
# impute with MEAN: you can package as many imputation columns as needed in one builder
impute_mean_X2 = dprep.ImputeColumnArguments(column_id='X2',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_mean_X2_lag_1 = dprep.ImputeColumnArguments(column_id='X2_lag_1',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_mean_X2_lag_7 = dprep.ImputeColumnArguments(column_id='X2_lag_7',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_mean_X2_rolling_7 = dprep.ImputeColumnArguments(column_id='X2_rolling_7',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_builder = d_ts_features_dflow.builders.impute_missing_values(
    impute_columns=[impute_mean_X2,impute_mean_X2_lag_1,impute_mean_X2_lag_7,impute_mean_X2_rolling_7])
impute_builder.learn()
d_ts_features_dflow = impute_builder.to_dataflow()

d_ts_features_dflow.to_pandas_dataframe().head(10)

,RDATE,X1,X1_TEMP,X1_TEMP_MEDIUM,X1_TEMP_HOT,X1_TEMP_COLD,X2,X2_lag_1,X2_lag_7,X2_rolling_7
0,2012-01-01,59.8,COLD,0,0,1,47365.60510,48127.07006,89002.86624,NaN
1,2012-01-02,62.0,MEDIUM,1,0,0,48127.07006,54601.27389,59700.95541,NaN
2,2012-01-03,62.0,MEDIUM,1,0,0,54601.27389,74656.36943,55801.91083,NaN
3,2012-01-04,63.4,MEDIUM,1,0,0,74656.36943,87156.68790,64844.58599,NaN
4,2012-01-05,55.4,COLD,0,0,1,87156.68790,100811.78340,69787.89809,NaN
5,2012-01-06,54.8,COLD,0,0,1,100811.78340,75496.49682,37906.68790,NaN
6,2012-01-07,55.0,COLD,0,0,1,75496.49682,89002.86624,44400.00000,69745.040943
7,2012-01-08,58.6,COLD,0,0,1,89002.86624,59700.95541,49031.21019,75693.221106
8,2012-01-09,57.6,COLD,0,0,1,59700.95541,55801.91083,50068.15287,77346.633299
9,2012-01-10,58.8,COLD,0,0,1,55801.91083,64844.58599,52394.58599,77518.152861


## FINAL azureml-dataprep-sdk.py Script

In [43]:
# =======================
# azureml-dataprep-sdk.py
# =======================

h_ts_1_dflow = dprep.auto_read_file(h_ts_1_dr)
h_ts_2_dflow = dprep.auto_read_file(h_ts_2_dr)
h_ts_3_dflow = dprep.auto_read_file(h_ts_3_dr)
d_ts_1_dflow = dprep.auto_read_file(d_ts_1_dr)
d_ts_2_dflow = dprep.auto_read_file(d_ts_2_dr)

# Pivot data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_2_pivot_dflow = h_ts_2_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])

# merge DATE and HOUR and update datatype for DATETIME column
def ts_merge_date_hour_to_datetime(dflow, date_column_name, hour_column_name):
    # merge columns
    dflow = dflow.derive_column_by_example(
        source_columns = [date_column_name,hour_column_name],
        new_column_name = 'DATETIME',
        example_data = [({date_column_name: '1/1/2012', hour_column_name: '1'},    '01/01/2012 01:00'),
                        ({date_column_name: '10/10/2012', hour_column_name: '15'}, '10/10/2012 15:00'),
                        ({date_column_name: '1/17/2012', hour_column_name: '12'},  '01/17/2012 12:00')]
        ).drop_columns([hour_column_name])
    # update data type
    builder = dflow.builders.set_column_types()
    builder.learn()
    builder.conversion_candidates[date_column_name] = (dprep.FieldType.DATE, ['%m/%d/%Y'])
    builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%m/%d/%Y %H:%M'])
    return builder.to_dataflow()

# generate all DATETIME columns with proper data type
h_ts_1_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_1_pivot_dflow,'MYDATE','HOUR')
h_ts_2_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_2_pivot_dflow,'MYDATE','HOUR')
h_ts_3_dt_dflow =       ts_merge_date_hour_to_datetime(h_ts_3_dflow,      'DATE','HE')

# JOINING DATA SET h_ts_dflow=join(h1,h2,h3)
h_ts_dflow = dprep.Dataflow.join(
    dprep.Dataflow.join(h_ts_1_pivot_dt_dflow,
                        h_ts_2_pivot_dt_dflow,
                        join_key_pairs=[('DATETIME', 'DATETIME')],
                        left_column_prefix='h1_',right_column_prefix='h2_'
                       ).drop_columns(['h2_MYDATE','h2_DATETIME']).rename_columns({'h1_MYDATE':'MYDATE','h1_DATETIME':'DATETIME'}),
    h_ts_3_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='',
    right_column_prefix='h3_'
).drop_columns(['h3_DATE','h3_DATETIME'])

# JOINING DATA SET d_ts_dflow=join(d1,d2)
d_ts_dflow = dprep.Dataflow.join(
    d_ts_1_dflow,
    d_ts_2_dflow,
    join_key_pairs=[('RDATE', 'RDATE')],
    left_column_prefix='',
    right_column_prefix='r_').drop_columns(['r_RDATE']).rename_columns({'r_X2':'X2'})

# helper: generate summary column
def generate_summary_column(column_name,column_suffix,summary_function):
    return dprep.SummaryColumnsValue(
                column_id=column_name,
                summary_column_name=column_name+'_'+column_suffix,
                summary_function=summary_function)

# helper: generate summary column for a few functions for each column that's not DATETIME
def generate_summary_columns(dflow):
    summary_columns = []
    for key in h_ts_dflow.get_profile().columns.keys():
        if key != 'MYDATE' and key != 'DATETIME':
            summary_columns.append(generate_summary_column(key,'MAX',dprep.SummaryFunction.MAX))
            summary_columns.append(generate_summary_column(key,'MIN',dprep.SummaryFunction.MIN))
            summary_columns.append(generate_summary_column(key,'MEAN',dprep.SummaryFunction.MEAN))
            summary_columns.append(generate_summary_column(key,'MEDIAN',dprep.SummaryFunction.MEDIAN))
    return summary_columns

# summarize h_ts_dflow to daily
h_ts_summarized_dflow = h_ts_dflow.summarize(
    summary_columns=generate_summary_columns(h_ts_dflow),
    group_by_columns=['MYDATE'])

# FEATURIZATION of X2

# X2: replace X2 values with quantile values for < 5%, > 95%
X2_column_profile = d_ts_dflow.get_profile().columns['X2']
X2_quantile_005 = X2_column_profile.quantiles[0.05]
X2_quantile_095 = X2_column_profile.quantiles[0.95]
# mapping function to update the dataflow, this gets executed in parallel for each dataflow partition
def update_with_quantiles(df, index):
    df.loc[df['X2'] < X2_quantile_005, 'X2'] = X2_quantile_005
    df.loc[df['X2'] > X2_quantile_095, 'X2'] = X2_quantile_095
    return df
d_ts_features_dflow = d_ts_dflow.map_partition(fn=update_with_quantiles)

# X1: create an 'X1_TEMP' feature
# we can use custom script to code any type of logic to generate new columns, but this is slower than the 'map_partition
# above which can run in parallel when multiple partitions are applied
d_ts_features_dflow = d_ts_features_dflow.new_script_column(new_column_name='X1_TEMP', insert_after='X1', script="""
def newvalue(row):
    value = 'MEDIUM'
    if row['X1'] > 80:
        value = 'HOT'
    if row['X1'] < 60:
        value = 'COLD'
    return value
""")

# X1: One Hot Encoding of 'X1_TEMP'
d_ts_features_dflow = d_ts_features_dflow.one_hot_encode(source_column='X1_TEMP')

# X2: mapping function to update the dataflow with Lags
def update_X2_with_lags(df, index):
    df['X2_lag_1'] = df['X2'].shift(-1)
    df['X2_lag_7'] = df['X2'].shift(-7)
    df['X2_rolling_7'] = df['X2'].rolling(7).mean()
    return df
d_ts_features_dflow = d_ts_features_dflow.map_partition(fn=update_X2_with_lags)

# X2 (and X2 lags): impute with MEAN: you can package as many imputation columns as needed in one builder
impute_mean_X2 = dprep.ImputeColumnArguments(column_id='X2',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_mean_X2_lag_1 = dprep.ImputeColumnArguments(column_id='X2_lag_1',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_mean_X2_lag_7 = dprep.ImputeColumnArguments(column_id='X2_lag_7',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_mean_X2_rolling_7 = dprep.ImputeColumnArguments(column_id='X2_rolling_7',impute_function=dprep.ReplaceValueFunction.MEAN)
impute_builder = d_ts_features_dflow.builders.impute_missing_values(
    impute_columns=[impute_mean_X2,impute_mean_X2_lag_1,impute_mean_X2_lag_7,impute_mean_X2_rolling_7])
impute_builder.learn()
d_ts_features_dflow = impute_builder.to_dataflow()

# join h and d series
training_dflow = dprep.Dataflow.join(
    h_ts_summarized_dflow,
    d_ts_features_dflow,
    join_key_pairs=[('MYDATE', 'RDATE')],
    left_column_prefix='',
    right_column_prefix='r_').drop_columns(['r_RDATE']).rename_columns(
        {'MYDATE':'DATETIME',
         'r_X1':'X1','r_X2':'X2',
         'r_X1_TEMP':'X1_TEMP',
         'r_X1_TEMP_COLD':'X1_TEMP_COLD','r_X1_TEMP_MEDIUM':'X1_TEMP_MEDIUM','r_X1_TEMP_HOT':'X1_TEMP_HOT',
         'r_X2_lag_1':'X2_lag_1','r_X2_lag_7':'X2_lag_7','r_X2_rolling_7':'X2_rolling_7'
        })

# ===========================
# EOF azureml-dataprep-sdk.py
# ===========================

In [44]:
# check head
training_dflow.head(10)

,DATETIME,h1_N1_MAX,h1_N1_MIN,h1_N1_MEAN,h1_N1_MEDIAN,h1_S1_MAX,h1_S1_MIN,h1_S1_MEAN,h1_S1_MEDIAN,h1_Z1_MAX,...,h3_CLOAD_MEDIAN,X1,X1_TEMP,X1_TEMP_MEDIUM,X1_TEMP_HOT,X1_TEMP_COLD,X2,X2_lag_1,X2_lag_7,X2_rolling_7
0,2012-01-01,77.964809,2.776551,21.051649,3.091959,783.017506,-5.092978,222.621629,-3.729854,228.803038,...,7235.987261,59.8,COLD,0,0,1,47365.60510,48127.07006,89002.86624,NaN
1,2012-01-02,55.436481,3.040615,15.320979,3.084408,568.235076,-5.019124,153.208645,-3.756280,210.288054,...,7326.751592,62.0,MEDIUM,1,0,0,48127.07006,54601.27389,59700.95541,NaN
2,2012-01-03,70.889146,3.056631,17.834587,3.153653,493.581704,-4.856099,125.727916,-3.735742,140.487076,...,7118.152866,62.0,MEDIUM,1,0,0,54601.27389,74656.36943,55801.91083,NaN
3,2012-01-04,39.108446,3.045873,10.545203,3.152172,281.776172,-4.759627,60.874459,-3.723787,153.746898,...,8285.668790,63.4,MEDIUM,1,0,0,74656.36943,87156.68790,64844.58599,NaN
4,2012-01-05,50.598615,3.050436,9.410610,3.281084,138.688640,-4.402857,24.784628,-3.940164,106.742688,...,8669.426752,55.4,COLD,0,0,1,87156.68790,100811.78340,69787.89809,NaN
5,2012-01-06,63.747131,3.205822,14.104283,3.233854,459.298678,-4.688280,114.269030,-3.818143,177.649086,...,8805.095541,54.8,COLD,0,0,1,100811.78340,75496.49682,37906.68790,NaN
6,2012-01-07,75.660309,3.127787,21.082898,3.196659,495.259169,-5.234389,101.290197,-3.414150,185.476666,...,8396.656051,55.0,COLD,0,0,1,75496.49682,89002.86624,44400.00000,69745.040943
7,2012-01-09,61.936949,3.340841,14.772332,3.674479,814.336490,-6.486188,209.323265,-5.278658,77.274029,...,7571.496816,57.6,COLD,0,0,1,59700.95541,55801.91083,50068.15287,77346.633299
8,2012-01-10,68.557357,3.452258,18.315124,3.541989,696.400790,-6.712863,156.085049,-5.232861,78.915102,...,7139.331210,58.8,COLD,0,0,1,55801.91083,64844.58599,52394.58599,77518.152861
9,2012-01-11,91.834946,0.355290,25.447751,3.477470,990.411955,-6.604443,273.632990,-5.252368,209.934016,...,8033.439491,60.8,MEDIUM,1,0,0,64844.58599,69787.89809,53425.15924,76116.469513


In [45]:
# check profile
training_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
DATETIME,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,,,,,,,,,,,,,,,,
h1_N1_MAX,FieldType.DECIMAL,-1.04211,473.713,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-0.919621,92.9141,92.9141,187.567,266.448,366.609,435.059,451.472,470.646,261.127,122.059,14898.5,-0.391051,-0.583275,473.713,-1.04211
h1_N1_MIN,FieldType.DECIMAL,-32.7938,7.91993,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-11.5086,-1.14324,-1.14448,-1.03392,-0.819506,-0.563383,0.92885,3.17949,7.21483,-0.662073,1.34198,1.80091,-9.37838,231.367,0.0983457,-1.59163
h1_N1_MEAN,FieldType.DECIMAL,-1.04211,216.837,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-0.919621,20.1795,20.1795,55.9374,88.2134,139.906,194.144,209.546,216.536,95.5194,56.1388,3151.57,0.203564,-0.780106,216.837,-1.04211
h1_N1_MEDIAN,FieldType.DECIMAL,-10.7138,231.293,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-1.16002,-0.850363,-0.856266,-0.59386,4.10531,58.5485,161.104,190.998,203.929,35.0798,52.1428,2718.87,1.52575,1.3622,146.693,-10.7138
h1_S1_MAX,FieldType.DECIMAL,-9.05072,2437.81,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-8.63159,780.399,751.413,1339.54,1616.75,2015.6,2320.93,2399.84,2430.05,1564.32,591.614,350007,-1.08856,0.984294,2437.81,449.78
h1_S1_MIN,FieldType.DECIMAL,-14.5467,16.8798,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-14.393,-8.95739,-8.97564,-8.1452,-6.87572,-3.96793,-1.2179,3.41376,12.9617,-6.06113,3.00728,9.04376,1.3251,4.11493,1.91641,-14.4093
h1_S1_MEAN,FieldType.DECIMAL,-9.05072,1057.36,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-8.63159,206.189,189.262,413.541,562.5,740.21,953.499,1012.65,1047.78,553.941,255.41,65234.5,-0.3935,-0.267577,1057.36,-9.05072
h1_S1_MEDIAN,FieldType.DECIMAL,-10.204,781.353,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-9.85716,-6.8024,-6.83799,-2.84959,28.6368,272.201,590.398,688.812,762.836,148.798,203.337,41345.9,1.20022,0.215925,682.492,-10.204
h1_Z1_MAX,FieldType.DECIMAL,-1.62555,563.445,1505.0,0.0,1505.0,0.000000,0.0,0.0,>1000,-1.56783,166.223,156.606,291.913,378.888,463.764,526.188,552.693,562.103,357.454,139.679,19510.3,-1.02009,0.616074,563.445,44.316


In [46]:
# test output
training_dflow.write_to_csv(directory_path='training_dflow.csv').run_local()

## Creating RunConfiguration

In [47]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify dependencies
#run_config.environment.python.conda_dependencies = CondaDependencies.create(
#    conda_packages=['pandas'],
#    pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]', 'azureml-train-automl'], 
#    pin_sdk_version=False)
run_config.environment.python.conda_dependencies = CondaDependencies(
    conda_dependencies_file_path='data-prep-pipeline.yml')

#
print("== Run Configuration created")

== Run Configuration created


## Creating Pipeline

In [48]:
# The best practice is to use separate folders for scripts and its dependent files
# for each step and specify that folder as the source_directory for the step.
# This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted).
# Since changes in any files in the source_directory would trigger a re-upload of the snapshot, this helps
# keep the reuse of the step when there are no changes in the source_directory of the step.
source_directory_dataprep = 'src/azureml-dataprep-sdk'

# inputs
h_ts_1_dr = DataReference(datastore=blob_store,data_reference_name="h_ts_1",path_on_datastore="datasets/time-series/S_ACTUALS.csv")
h_ts_2_dr = DataReference(datastore=blob_store,data_reference_name="h_ts_2",path_on_datastore="datasets/time-series/W_ACTUALS.csv")
h_ts_3_dr = DataReference(datastore=blob_store,data_reference_name="h_ts_3",path_on_datastore="datasets/time-series/C_LOAD.csv")
d_ts_1_dr = DataReference(datastore=blob_store,data_reference_name="d_ts_1",path_on_datastore="datasets/time-series/X1.csv")
d_ts_2_dr = DataReference(datastore=blob_store,data_reference_name="d_ts_2",path_on_datastore="datasets/time-series/X2.csv")

# output
d_use_case_1_dataprep_sdk_pd = PipelineData("d_use_case_1_dataprep_sdk",datastore=blob_store)

# Step
use_case_1_dataprep_sdk_step = PythonScriptStep(
    script_name="azureml-dataprep-sdk.py",
    arguments=[ "--h_ts_1", h_ts_1_dr, "--h_ts_2", h_ts_2_dr, "--h_ts_3", h_ts_3_dr,
                "--d_ts_1", d_ts_1_dr, "--d_ts_2", d_ts_2_dr,
                "--output", d_use_case_1_dataprep_sdk_pd ],
    inputs=[h_ts_1_dr,h_ts_2_dr,h_ts_3_dr,
            d_ts_1_dr,d_ts_2_dr],
    outputs=[d_use_case_1_dataprep_sdk_pd],
    compute_target=aml_compute,
    source_directory=source_directory_dataprep,
    runconfig=run_config
)

print("== PythonScriptStep use_case_1_dataprep_step created")

== PythonScriptStep use_case_1_dataprep_step created


### Build the pipeline and submit an Experiment run

In [49]:
pipeline = Pipeline(workspace=ws, steps=[use_case_1_dataprep_sdk_step])
print ("== Pipeline is built")

== Pipeline is built


In [ ]:
pipeline_run = Experiment(ws, 'use-case-1-dataprep-sdk').submit(pipeline)
print("== Pipeline is submitted for execution")

In [51]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

### Wait for pipeline run to complete

In [ ]:
pipeline_run.wait_for_completion(show_output=True)

### See Outputs

See where outputs of each pipeline step are located on your datastore.

***Wait for pipeline run to complete, to make sure all the outputs are ready***

In [53]:
# Get Steps
for step in pipeline_run.get_steps():
    print("== Outputs of step " + step.name)
    
    # Get a dictionary of StepRunOutputs with the output name as the key 
    output_dict = step.get_outputs()
    
    for name, output in output_dict.items():
        output_reference = output.get_port_data_reference() # Get output port data reference
        print("\tname: " + name)
        print("\tdatastore: " + output_reference.datastore_name)
        print("\tpath on datastore: " + output_reference.path_on_datastore)

== Outputs of step azureml-dataprep-sdk.py
	name: d_use_case_1_dataprep_sdk
	datastore: workspaceblobstore
	path on datastore: azureml/1ffa8568-4425-4c0d-aeb1-a66c8ba024a8/d_use_case_1_dataprep_sdk


In [54]:
# REGISTER a new version of the final output as a Dataset

from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

# find output dataset
for step in pipeline_run.get_steps():
    output_dict = step.get_outputs()
    for name, output in output_dict.items():
        if name == 'd_use_case_1_dataprep_sdk':
            # generate a Tabular DataSet for it
            output_reference = output.get_port_data_reference()
            datastore_path = [DataPath(blob_store, output_reference.path_on_datastore)]
            ds = Dataset.Tabular.from_delimited_files(datastore_path)
            dataset_name = 'd_use_case_1_dataprep_sdk'
            ds.register(ws, name=dataset_name, create_new_version=True)
            print("== Registered new version of dataset: " + dataset_name)

== Registered new version of dataset: d_use_case_1_dataprep_sdk
